<a href="https://colab.research.google.com/github/dquerales/jupyter-automation-github-actions/blob/main/notebooks/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Training
Daniel Querales - d.querales@gmail.com

___

 **Changelog:**
- 2023-01-02: File created.

___

## Table of Contents
- [Importing libraries](#Importing-libraries)
- [Load data](#Load-data)
- [Data Cleaning and Preparation](#Data-Cleaning-and-Preparation)
- [Data Exploration & Visualization](#Data-Exploration-&-Visualization)

---

## Importing libraries

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split

____

## Load data

In [ ]:
df = pd.read_csv('../datasets/raw/data_raw.csv')

In [ ]:
df.head()

In [ ]:
df.shape

____

## Data Preparation


In [ ]:
def add_date_features(dataframe, date):
    df = dataframe.copy()
    df[date] = pd.to_datetime(df[date])
    df['hour'] = df[date].dt.hour
    df['dayofweek'] = df[date].dt.dayofweek
    df['quarter'] = df[date].dt.quarter
    df['month'] = df[date].dt.month
    df['year'] = df[date].dt.year
    df['dayofyear'] = df[date].dt.dayofyear
    df['dayofmonth'] = df[date].dt.day
    df = df.set_index(date).sort_index()
    df['series'] = np.arange(1,len(df)+1)
    return(df)

In [ ]:
df = add_date_features(df, 'date')

In [ ]:
df.head()

In [ ]:
TARGET = 'price'

In [ ]:
X = df.drop([TARGET], axis=1)
y = df[TARGET]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

____

## Modelling

In [ ]:
model = xgb.XGBRegressor(n_estimators=1000)
model.fit(X_train, y_train)

In [ ]:
model.save_model("../models/model.json")